In [ ]:
import cv2
import numpy as np

# Load HAAR face classifier
face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# Load functions
def face_extractor(img):
    # Function detects faces and returns the cropped face
    # If no face detected, it returns the input image
    
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    
    if faces is ():
        return None
    
    # Crop all faces found
    for (x,y,w,h) in faces:
        cropped_face = img[y:y+h, x:x+w]

    return cropped_face

# Initialize Webcam
cap = cv2.VideoCapture(0)
count = 0

# Collect 100 samples of your face from webcam input
while True:

    ret, frame = cap.read()
    if face_extractor(frame) is not None:
        count += 1
        face = cv2.resize(face_extractor(frame), (200, 200))
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

        # Save file in specified directory with unique name
        file_name_path = './faces/user/' + str(count) + '.jpg'
        cv2.imwrite(file_name_path, face)

        # Put count on images and display live count
        cv2.putText(face, str(count), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
        cv2.imshow('Face Cropper', face)
        
    else:
        print("Face not found")
        pass

    if cv2.waitKey(1) == 13 or count == 100: #13 is the Enter Key
        break
        
cap.release()
cv2.destroyAllWindows()      
print("Collecting Samples Complete")

In [1]:
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join

# Get the training data we previously made
data_path = './faces/user/'
onlyfiles = [f for f in listdir(data_path) if isfile(join(data_path, f))]

# Create arrays for training data and labels
Training_Data, Labels = [], []

# Open training images in our datapath
# Create a numpy array for training data
for i, files in enumerate(onlyfiles):
    image_path = data_path + onlyfiles[i]
    images = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    Training_Data.append(np.asarray(images, dtype=np.uint8))
    Labels.append(i)

# Create a numpy array for both training data and labels
Labels = np.asarray(Labels, dtype=np.int32)

# Initialize facial recognizer
# model = cv2.face.createLBPHFaceRecognizer()
# NOTE: For OpenCV 3.0 use cv2.face.createLBPHFaceRecognizer()
# pip install opencv-contrib-python
# model = cv2.createLBPHFaceRecognizer()

vimal_model  = cv2.face_LBPHFaceRecognizer.create()
# Let's train our model 
vimal_model.train(np.asarray(Training_Data), np.asarray(Labels))
print("Model trained sucessefully")


Model trained sucessefully


In [2]:
import cv2
import numpy as np
import os
import pywhatkit
import datetime
import smtplib

def send_mail():
    server = smtplib.SMTP_SSL("smtp.gmail.com", 465)
    server.login("krishrj56@gmail.com", "Abcd97@4321")
    server.sendmail("krishrj56@gmail.com", "krishna420bhore@gmail.com", 'Hey There!')
    server.quit()
    
    
def whatsapp():
    now = datetime.datetime.now()
    Hrs = int(now.strftime("%H"))
    Min = int(now.strftime("%M"))+1
    print("Current =", Hrs,Min)
    pywhatkit.sendwhatmsg('+919009078504', 'Some random message that you should igmore...',Hrs,Min,1)


def face_detector(img, size=0.5):
    
    # Convert image to grayscale
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    if faces is ():
        return img, []
    
    
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
        roi = img[y:y+h, x:x+w]
        roi = cv2.resize(roi, (200, 200))
    return img, roi

face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# Open Webcam
cap = cv2.VideoCapture(0)

while True:

    ret, frame = cap.read()
    
    image, face = face_detector(frame)
    
    try:
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

        # Pass face to prediction model
        # "results" comprises of a tuple containing the label and the confidence value
        results = vimal_model.predict(face)
        # harry_model.predict(face)
        
        if results[1] < 500:
            confidence = int( 100 * (1 - (results[1])/400) )
            display_string = str(confidence) + '% Confident it is User'
            
        cv2.putText(image, display_string, (100, 120), cv2.FONT_HERSHEY_COMPLEX, 1, (255,120,150), 2)
        
        if confidence >80:
            cv2.putText(image, "Hey Krishna", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
            cv2.imshow('Face Recognition', image )    
            cap.release()
            cv2.destroyAllWindows()    
            send_mail()    #Mail Sender 
            
            #creating instance
            os.system("aws ec2 run-instances --image-id ami-0ad704c126371a549 --count 1 --instance-type t2.micro --key-name test1 --security-groups mysg   --tag-specifications ResourceType=instance,Tags=[{Key=Name,Value=ins_8}]")
            #creating volume
            os.system("aws ec2 create-volume --availability-zone ap-south-1a --volume-type gp2 --size 5 --tag-specifications ResourceType=volume,Tags=[{Key=Name,Value=volume_15}]")
            #attachine volume and Instance
            ins_id=os.popen("aws ec2 describe-instances --filters Name=tag:Name,Values=ins_8 --query Reservations[].Instances[].InstanceId --output text").read()
            print("instance ID " + ins_id)
            vol_id=os.popen("aws ec2 describe-volumes --filters Name=status,Values=available --query Volumes[].VolumeId --output text").read()
            print("volume ID " + vol_id)
            while True:
                state=os.popen("aws ec2 describe-instances  --filters  --instance-ids " + ins_id.replace("\n", "") +" --query Reservations[*].Instances[*].[State.Name] --o text").read()
                if (state.replace("\n", "")) == "running":
                    os.popen(f"aws ec2 attach-volume --volume-id " + vol_id.replace("\n", "") + " --instance-id " + ins_id.replace("\n", "") + " --device /dev/sdf").read()
                    print("Volume successfully attach")
                    break
                else:
                    continue
            break
            whatsapp()    #Whatsapp Msg Sender
        else: 
            
            cv2.putText(image, "I dont know, how r u", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
            cv2.imshow('Face Recognition', image )

    except:
        cv2.putText(image, "No Face Found", (220, 120) , cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.putText(image, "looking for face", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.imshow('Face Recognition', image )
        pass
        
    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break
        
cap.release()
cv2.destroyAllWindows()  

<>:28: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:28: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-2-1bedf015e6e3>:28: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if faces is ():


instance ID i-083e96417ae350e38

volume ID vol-01bf9a98f16763680

Volume successfully attach


In [ ]:
cap.release()

Current = 18 28
In 10 seconds web.whatsapp.com will open and after 1 seconds message will be delivered


ValueError: sleep length must be non-negative